In [7]:
import pandas as pd
import os

dataset_name = 'jp'

for seed in range(6):
    ans = []
    for turn in range(5):
        if os.path.exists(f'results/{dataset_name}/seed{seed}/result{turn}.parquet'):
            temp = pd.read_parquet(f'results/{dataset_name}/seed{seed}/result{turn}.parquet')
            temp.columns = ['job_title', 'job_description', f'task_seed{seed}']
            temp[f'task_seed{seed}'] = temp[f'task_seed{seed}'].apply(lambda x: [x[i] for i in range(len(x)) if x[i] not in x[:i]])
            ans.append(temp)
    if len(ans) == 0:
        continue
    ans = pd.concat(ans)
    if seed == 0:
        final_task = ans
    else:
        final_task = pd.merge(final_task, ans, on=['job_title', 'job_description'])
final_task

,job_title,job_description,task_seed0,task_seed1,task_seed2,task_seed3,task_seed5
0,【プロダクトマネージャー】/インターネット/Webサービス・ASP,まちの回遊性向上を図れるデジタルスタンプラリーやデジタルクーポン機能、地域経済を活性化させる...,"[システム操作の図やフローチャートを作成します。, 技術文書を確認して作業を計画します。, ...","[緊急要員を要請します。, システム操作の図やフローチャートを作成します。, メンテナンスや...","[部門間または他のグループ間の連絡を取り、機能やコミュニケーションを改善します。, 健康関連...","[システム操作の図やフローチャートを作成します。, 運用活動を文書化します。, 環境管理や保...","[部門間または他のグループ間の連絡を取り、機能やコミュニケーションを改善します。, 犯罪を解..."
1,Go/BaaSプロダクトの開発,プロダクトマネージャが提示するプロダクトのビジョンや要件を、主に技術的な観点からブレイクダウ...,"[ニュース記事のための情報を収集します。, ビジネス戦略、実践、またはポリシーについてマネー...","[ビジネス戦略または市場戦略を策定します。, 財務情報の正確性を確認します。, 当事者間の紛...","[人事ポリシーまたは人事慣行の有効性を評価します。, 人事に関するトピックについて他の人にア...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, 財務情報を分析し...","[法的手続きに関連する文書情報。, 必要な情報を取得するために、ファイル、データベース、また..."
2,【共創プロダクトマネージャー】マネーフォワードエックス_東京,様々な局面から参画いただくことがありますが、最大のスコープとしては、営業支援から始まり、企画...,"[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, ビジネス戦略また...","[ビジネス戦略または市場戦略を策定します。, プロジェクト情報を他の人に伝えます。, ビジネ...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, ビジネスまたは財...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, ビジネス戦略また...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, マーケティング計..."
3,デザイン業務(グラフィックデザイン、DTPデザイン、webデザイン)(石川),ロゴやパッケージ、会社案内、社内報、広告デザイン、カタログ/リーフレットなどの企画からDTP...,"[組織のプロセスまたはポリシーの変更を推奨します。, コンピュータまたは情報システムを改善す...","[販促資料を作成します。, 組織のプロセスまたはポリシーの変更を推奨します。, プレゼンテー...","[販促資料を作成します。, 組織のプロセスまたはポリシーの変更を推奨します。, 環境問題を特...","[組織のプロセスまたはポリシーの変更を推奨します。, プレゼンテーションの主題または内容を決...","[組織のプロセスまたはポリシーの変更を推奨します。, プレゼンテーションの主題または内容を決..."
4,地図データ入力作業者,空中写真から地形や地物を読み取る面白い仕事です！継続的なスキルアップは必要になりますが、みん...,"[組織のメンバーと協議して業務を遂行します。, 技術担当者と協議して設計や運用計画を作成しま...","[部門間または他のグループ間の連絡を取り、機能やコミュニケーションを改善します。, 残骸を調...","[手順に従って法医学的または法的証拠を処理します。, 水文学的特徴またはプロセスを研究します...","[メンテナンスや修理を行うために必要な技術情報をお読みください。, 重要な統計やその他の健康...","[計画や仕様を評価して、技術的または環境的影響を判断します。, グリーンテクノロジーの設計要..."
...,...,...,...,...,...,...,...
2727,シニアプロダクトマネージャー/モバイル/アプリサービス,KPI達成のための事業推進,"[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, 調査レポートまた...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, 法的手続きに関連...","[ビジネス戦略または市場戦略を策定します。, 財務記録を検査して、ポリシーや規制に準拠してい...","[ビジネス戦略または市場戦略を策定します。, 財務情報の正確性を確認します。, 法律または規...","[ビジネス戦略または市場戦略を策定します。, マーケティング計画や戦略を策定します。, 財務..."
2728,音響設計エンジニア,製品の音響仕様の定義をする,"[新しい製品またはプロセスの仕様を開発します。, 設計基準または仕様を決定します。, 製品の...","[新しい製品またはプロセスの仕様を開発します。, 設計基準または仕様を決定します。, 製品の...","[新しい製品またはプロセスの仕様を開発します。, 機器や備品を清潔にします。, 製品の機能や...","[新しい製品またはプロセスの仕様を開発します。, 施設、設備、または備品を検査して、基準に準...","[新しい製品またはプロセスの仕様を開発します。, グリーンエネルギー機器、システム、または施..."
2729,Webデザイナー,メールマガジンの編集,"[ビジネス戦略または市場戦略を策定します。, 情報を明確にするために裁判所の職員と協議してく...","[マーケティング計画や戦略を策定します。, 顧客の口座から債権者に資金を支払います。, 新興...","[ニュースソースから得た情報を分析します。, 人事に関するトピックについて他の人にアドバイス...","[マーケティング活動を実行します。, 他のユーザーと協力してマーケティング戦略を開発または実...","[顧客の口座から債権者に資金を支払います。, 紛争を調停する。, 観察、インタビュー、調査な..."
2730,プロダクトマネージャー/その他金融・保険系,デザイナーとの連携,"[マーケティング活動を実行します。, マーケティング活動において他者と協力します。, 人事ポ...","[情報資料や参考資料を準備します。, 人事ポリシーまたは人事慣行の有効性を評価します。, ト...","[ウェブサイトやウェブアプリケーションを設計します。, 教育プログラムまたはトレーニング プ...","[人事ポリシーまたは人事慣行の有効性を評価します。, 環境条件に関する情報を記録します。, ...","[ビジネス戦略または市場戦略を策定します。, 歴史研究を実施する。, 他の人のために情報を翻..."


In [8]:
origin_df = pd.read_csv(f'../step3-bert_recall/CuBERT/results/{dataset_name}/2e-05_32_0.8/{dataset_name}.csv', sep='\t')
origin_df['sim'] = origin_df.apply(lambda x: sum([x[f"task{i}_sim"] for i in range(100)]), axis=1)
origin_df = origin_df[['job_title', 'job_description', 'sim']]
final_task = pd.merge(final_task, origin_df, on=['job_title', 'job_description'])
final_task

,job_title,job_description,task_seed0,task_seed1,task_seed2,task_seed3,task_seed5,sim
0,【プロダクトマネージャー】/インターネット/Webサービス・ASP,まちの回遊性向上を図れるデジタルスタンプラリーやデジタルクーポン機能、地域経済を活性化させる...,"[システム操作の図やフローチャートを作成します。, 技術文書を確認して作業を計画します。, ...","[緊急要員を要請します。, システム操作の図やフローチャートを作成します。, メンテナンスや...","[部門間または他のグループ間の連絡を取り、機能やコミュニケーションを改善します。, 健康関連...","[システム操作の図やフローチャートを作成します。, 運用活動を文書化します。, 環境管理や保...","[部門間または他のグループ間の連絡を取り、機能やコミュニケーションを改善します。, 犯罪を解...",99.584339
1,Go/BaaSプロダクトの開発,プロダクトマネージャが提示するプロダクトのビジョンや要件を、主に技術的な観点からブレイクダウ...,"[ニュース記事のための情報を収集します。, ビジネス戦略、実践、またはポリシーについてマネー...","[ビジネス戦略または市場戦略を策定します。, 財務情報の正確性を確認します。, 当事者間の紛...","[人事ポリシーまたは人事慣行の有効性を評価します。, 人事に関するトピックについて他の人にア...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, 財務情報を分析し...","[法的手続きに関連する文書情報。, 必要な情報を取得するために、ファイル、データベース、また...",98.971943
2,【共創プロダクトマネージャー】マネーフォワードエックス_東京,様々な局面から参画いただくことがありますが、最大のスコープとしては、営業支援から始まり、企画...,"[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, ビジネス戦略また...","[ビジネス戦略または市場戦略を策定します。, プロジェクト情報を他の人に伝えます。, ビジネ...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, ビジネスまたは財...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, ビジネス戦略また...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, マーケティング計...",98.845265
3,デザイン業務(グラフィックデザイン、DTPデザイン、webデザイン)(石川),ロゴやパッケージ、会社案内、社内報、広告デザイン、カタログ/リーフレットなどの企画からDTP...,"[組織のプロセスまたはポリシーの変更を推奨します。, コンピュータまたは情報システムを改善す...","[販促資料を作成します。, 組織のプロセスまたはポリシーの変更を推奨します。, プレゼンテー...","[販促資料を作成します。, 組織のプロセスまたはポリシーの変更を推奨します。, 環境問題を特...","[組織のプロセスまたはポリシーの変更を推奨します。, プレゼンテーションの主題または内容を決...","[組織のプロセスまたはポリシーの変更を推奨します。, プレゼンテーションの主題または内容を決...",98.919880
4,地図データ入力作業者,空中写真から地形や地物を読み取る面白い仕事です！継続的なスキルアップは必要になりますが、みん...,"[組織のメンバーと協議して業務を遂行します。, 技術担当者と協議して設計や運用計画を作成しま...","[部門間または他のグループ間の連絡を取り、機能やコミュニケーションを改善します。, 残骸を調...","[手順に従って法医学的または法的証拠を処理します。, 水文学的特徴またはプロセスを研究します...","[メンテナンスや修理を行うために必要な技術情報をお読みください。, 重要な統計やその他の健康...","[計画や仕様を評価して、技術的または環境的影響を判断します。, グリーンテクノロジーの設計要...",99.525600
...,...,...,...,...,...,...,...,...
2727,シニアプロダクトマネージャー/モバイル/アプリサービス,KPI達成のための事業推進,"[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, 調査レポートまた...","[ビジネス戦略、実践、またはポリシーについてマネージャーと話し合います。, 法的手続きに関連...","[ビジネス戦略または市場戦略を策定します。, 財務記録を検査して、ポリシーや規制に準拠してい...","[ビジネス戦略または市場戦略を策定します。, 財務情報の正確性を確認します。, 法律または規...","[ビジネス戦略または市場戦略を策定します。, マーケティング計画や戦略を策定します。, 財務...",98.723595
2728,音響設計エンジニア,製品の音響仕様の定義をする,"[新しい製品またはプロセスの仕様を開発します。, 設計基準または仕様を決定します。, 製品の...","[新しい製品またはプロセスの仕様を開発します。, 設計基準または仕様を決定します。, 製品の...","[新しい製品またはプロセスの仕様を開発します。, 機器や備品を清潔にします。, 製品の機能や...","[新しい製品またはプロセスの仕様を開発します。, 施設、設備、または備品を検査して、基準に準...","[新しい製品またはプロセスの仕様を開発します。, グリーンエネルギー機器、システム、または施...",99.946723
2729,Webデザイナー,メールマガジンの編集,"[ビジネス戦略または市場戦略を策定します。, 情報を明確にするために裁判所の職員と協議してく...","[マーケティング計画や戦略を策定します。, 顧客の口座から債権者に資金を支払います。, 新興...","[ニュースソースから得た情報を分析します。, 人事に関するトピックについて他の人にアドバイス...","[マーケティング活動を実行します。, 他のユーザーと協力してマーケティング戦略を開発または実...","[顧客の口座から債権者に資金を支払います。, 紛争を調停する。, 観察、インタビュー、調査な...",97.507494
2730,プロダクトマネージャー/その他金融・保険系,デザイナーとの連携,"[マーケティング活動を実行します。, マーケティング活動において他者と協力します。, 人事ポ...","[情報資料や参考資料を準備します。, 人事ポリシーまたは人事慣行の有効性を評価します。, ト...","[ウェブサイトやウェブアプリケーションを設計します。, 教育プログラムまたはトレーニング プ...","[人事ポリシーまたは人事慣行の有効性を評価します。, 環境条件に関する情報を記録します。, ...","[ビジネス戦略または市場戦略を策定します。, 歴史研究を実施する。, 他の人のために情報を翻...",97.979379


In [9]:
from utlis import aggregate_expert_opinions
final_task['task_top5'] = final_task.apply(lambda x: aggregate_expert_opinions([x[f'task_seed{i}'] for i in range(6) if f'task_seed{i}' in x], top_num=5)[0], axis=1)
final_task['task'] = final_task['task_top5'].apply(lambda x: x[0])
final_task['score'] = final_task.apply(lambda x: aggregate_expert_opinions([x[f'task_seed{i}'] for i in range(6) if f'task_seed{i}' in x], top_num=1)[1][0], axis=1)
final_ans = final_task[['job_title', 'job_description', 'task', 'sim', 'score', 'task_top5']]
final_ans = final_ans.sort_values(['score', 'sim'], ascending=[False, False])
final_ans.to_excel(f'results/{dataset_name}/{dataset_name}_results.xlsx', index=None)
final_ans

,job_title,job_description,task,sim,score,task_top5
1944,梱包・発送・データ入力等,商品（自動車部品）の梱包,製品を保管または出荷用に梱包します。,99.968524,50.0,"[製品を保管または出荷用に梱包します。, 供給ラインを生産設備またはツールに接続します。, ..."
868,品質保証(機械),製鋼メーカーの不具合に対しての対応、調査 ※業務に慣れた後は、,溶融金属から不純物を取り除きます。,99.964607,50.0,"[溶融金属から不純物を取り除きます。, 機器、材料、工具、構造物からへこみを取り除きます。,..."
1494,【生産本部】品質管理(機械加工)〈栃木ニコンプレシジョン品質保証部出向〉,製造現場で使用している計測器の運用管理,材料または物体の物理的または化学的特性を測定します。,99.962635,50.0,"[材料または物体の物理的または化学的特性を測定します。, 製品または材料の寸法を測定します。..."
804,品質管理職,自社及び海外工場で生産する製品の品質保証を担当して いただきます,材料または完成品の品質をテストします。,99.962267,50.0,"[材料または完成品の品質をテストします。, 材料または構造の特性をテストします。, 完成した..."
1808,品質保証・監査,新製品設計時の品質基準設定,機器またはコンポーネントの構成を開発します。,99.961365,50.0,"[機器またはコンポーネントの構成を開発します。, 資材や機器を降ろします。, 装置のコントロ..."
...,...,...,...,...,...,...
2491,機械学習エンジニア(AI)/モバイル/アプリサービス,自社サービスにおける機械学習を活用した機能開発や改善、データマネジメントをお任せします,専門知識を維持するために法律や規制を確認します。,98.225381,15.0,"[専門知識を維持するために法律や規制を確認します。, 顧客に投資を推奨します。, 書かれた資..."
263,DX推進エンジニア/システムインテグレータ・ソフトハウス,DX時代における新たなシステム基盤アーキテクチャ検討、システムグランドデザイン策定から設計実...,権限のない人物が制限区域に侵入するのを防ぎます。,99.673725,14.0,"[権限のない人物が制限区域に侵入するのを防ぎます。, 新しいテクノロジーの開発や使用について..."
2417,【在宅可】品質保証(課長候補)/精密機器業界,是処置展開,施設を検査して、セキュリティまたは安全規制に準拠していることを確認します。,99.537630,14.0,"[施設を検査して、セキュリティまたは安全規制に準拠していることを確認します。, 組織の目標ま..."
529,WEBデザイナー・コーダー/広告・デザイン・イベント,制作内容は、ランディングページから、既存ページの改修、バナーデザイン等多岐に渡ります,法的手続きのための書面による決定を準備します。,98.951334,14.0,"[法的手続きのための書面による決定を準備します。, 戦略的なビジネス投資機会を特定します。,..."
